# Confidence-aware IBCF MultiEval Example

Much is this structure and organization is borrowed from the Lenskit sample evaluation walkthrough

## Setup

In [1]:
import sys # set path of locally install lenskit_confidence module
sys.path.insert(0,'C:\\Users\\Name\\Documents\\GitHub\\lenskit_confidence') # Looks like this on my machine

In [2]:
from lenskit.metrics import predict
import pandas as pd
import matplotlib.pyplot as plt

from lenskit.batch_ca import MultiEval
from lenskit.algorithms_ca import item_knn_ca, Recommender # *not* user_knn
from lenskit import topn, datasets, batch_ca # *not* batch 
from lenskit import datasets
from lenskit.datasets import MovieLens
from lenskit import crossfold as xf
from lenskit import topn, util #, metrics
from lenskit.crossfold import partition_users, SampleN

Setting up a progress bar...

In [3]:
from tqdm.notebook import tqdm_notebook as tqdm
tqdm.pandas()

C:\Users\Jacob\anaconda3\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


Setup logging to the notebook...

In [4]:
util.log_to_notebook()

[   INFO] lenskit.util.log notebook logging configured


Pick a dataset to run...

In [5]:
data = MovieLens('../data/ml-1m')
#data = MovieLens('../data/ml-10m')
#data = MovieLens('../data/ml-20m')
#data = MovieLens('../data/jester') # with Jester cleaning, it's the same format a ML datasets, so the ML input function works

## Experiment

Run experiment and store output in the `my-eval` directory. 

We're not producing prediction, generating 10-item recommendation lists, and setting up 4 workers.

In [6]:
eval = MultiEval('my-eval', predict = False, recommend = 10, eval_n_jobs = 4)

We'll use 5-fold CV, partitioning users and putting 5 ratings per user in the test set.  

In [7]:
pairs = list(partition_users(data.ratings, 5, SampleN(5)))

[   INFO] lenskit.crossfold partitioning 1000209 rows for 6040 users into 5 partitions
[   INFO] lenskit.crossfold fold 0: selecting test ratings
[   INFO] lenskit.crossfold fold 0: partitioning training data
[   INFO] lenskit.crossfold fold 1: selecting test ratings
[   INFO] lenskit.crossfold fold 1: partitioning training data
[   INFO] lenskit.crossfold fold 2: selecting test ratings
[   INFO] lenskit.crossfold fold 2: partitioning training data
[   INFO] lenskit.crossfold fold 3: selecting test ratings
[   INFO] lenskit.crossfold fold 3: partitioning training data
[   INFO] lenskit.crossfold fold 4: selecting test ratings
[   INFO] lenskit.crossfold fold 4: partitioning training data


Add the dataset to MultiEval with `add_datasets`.

In [8]:
eval.add_datasets(pairs, name = 'ML1M') # give the added dataset a name

In [9]:
nhbr_range = [25] # We'll use just K=25 for our sample evaluation [10, 25, 50, 75]

Add the algorithms to MultiEval with `add_algorithms`; the three CIBCF options are listed

In [10]:
eval.add_algorithms([item_knn_ca.ItemItemCA(nnbrs = f, aggregate = 'average', 
                                            variance_estimator = 'standard-deviation-average') for f in nhbr_range], 
                    attrs = ['nnbrs'], name = 'ItemKNN-CA-Average') 

In [ ]:
eval.add_algorithms([item_knn_ca.ItemItemCA(nnbrs = f, aggregate = 'average', 
                                            variance_estimator = 'standard-deviation-jackknife-average') for f in nhbr_range], 
                    attrs = ['nnbrs'], name = 'ItemKNN-CA-JK-Average') 

In [ ]:
eval.add_algorithms([item_knn_ca.ItemItemCA(nnbrs = f, aggregate = 'average', 
                                            variance_estimator = 'standard-deviation-bootstrap-average') for f in nhbr_range], 
                    attrs = ['nnbrs'], name = 'ItemKNN-CA-BS-Average') 

Run the experiment...

In [11]:
eval.run(progress = tqdm)

[   INFO] lenskit.batch_ca._multi_ca starting run 1: ItemItemCA(nnbrs=25, msize=None) on ML1M:1
[   INFO] lenskit.batch_ca._multi_ca adapting ItemItemCA(nnbrs=25, msize=None) into a recommender
[   INFO] lenskit.batch_ca._multi_ca training algorithm ItemItemCA(nnbrs=25, msize=None) on 994169 ratings
[   INFO] lenskit.algorithms_ca.item_knn_ca [1.15s] made sparse matrix for 3706 items (994169 ratings from 6040 users)
[   INFO] lenskit.algorithms_ca.item_knn_ca [1.24s] computed means for 3706 items
[   INFO] lenskit.algorithms_ca.item_knn_ca [1.36s] normalized rating matrix columns
[   INFO] lenskit.algorithms_ca.item_knn_ca [1.37s] computing similarity matrix
[   INFO] lenskit.algorithms_ca.item_knn_ca [3.84s] splitting 3706 items (992541 ratings) into 4 blocks
[   INFO] numba.core.transforms finding looplift candidates
[   INFO] lenskit.algorithms_ca.item_knn_ca [12.41s] computed 7917466 similarities for 3706 items in 4 blocks
[   INFO] lenskit.algorithms_ca.item_knn_ca [12.52s] sortin

[   INFO] lenskit.algorithms_ca.item_knn_ca [4.48s] computed 7915512 similarities for 3706 items in 4 blocks
[   INFO] lenskit.algorithms_ca.item_knn_ca [4.58s] sorting similarity matrix with 7915512 entries
[   INFO] lenskit.algorithms_ca.item_knn_ca [4.84s] got neighborhoods for 3560 of 3706 items
[   INFO] lenskit.algorithms_ca.item_knn_ca [4.85s] computed 7915512 neighbor pairs
[   INFO] lenskit.algorithms_ca.item_knn_ca [5.33s] transposed matrix for optimization
[   INFO] lenskit.algorithms_ca.basic_ca trained unrated candidate selector for 994169 ratings
[   INFO] lenskit.batch_ca._multi_ca trained algorithm ItemItemCA(nnbrs=25, msize=None) in 5.51s
[   INFO] lenskit.batch_ca._multi_ca generating recommendations for 1208 users for TopN/ItemItemCA(nnbrs=25, msize=None)
[   INFO] lenskit.sharing.shm serialized TopN/ItemItemCA(nnbrs=25, msize=None) to 1396 pickle bytes with 17 buffers of 206201864 bytes
[   INFO] lenskit.util.parallel setting up ProcessPoolExecutor w/ 4 workers
[   

## Analyzing Results

We need to read in experiment outputs.

First the run metadata:

In [12]:
runs = pd.read_csv('my-eval/runs.csv')
runs.set_index('RunId', inplace = True)
runs.head() # a quick visual check

,DataSet,Partition,AlgoClass,AlgoStr,name,nnbrs,TrainTime,PredTime,RecTime
RunId,,,,,,,,,
1,ML1M,1,ItemItemCA,"ItemItemCA(nnbrs=25, msize=None)",ItemKNN-CA-Average,25,15.991513,NaN,52.159636
2,ML1M,2,ItemItemCA,"ItemItemCA(nnbrs=25, msize=None)",ItemKNN-CA-Average,25,5.469378,NaN,51.119131
3,ML1M,3,ItemItemCA,"ItemItemCA(nnbrs=25, msize=None)",ItemKNN-CA-Average,25,5.441524,NaN,52.023368
4,ML1M,4,ItemItemCA,"ItemItemCA(nnbrs=25, msize=None)",ItemKNN-CA-Average,25,5.512648,NaN,52.118117
5,ML1M,5,ItemItemCA,"ItemItemCA(nnbrs=25, msize=None)",ItemKNN-CA-Average,25,5.705166,NaN,52.571828


This describes each run - a data set, partition, and algorithm combination.  To evaluate, we need to get the actual recommendations, and combine them with this:

In [13]:
recs = pd.read_parquet('my-eval/recommendations.parquet')
recs.head()

,item,prediction,user,var,num_nbhr,rank,RunId
0,53,0.633304,3,0.731744,25,1,1
1,2839,0.556983,3,0.658207,25,2,1
2,2503,0.543428,3,0.856513,25,3,1
3,904,0.444571,3,1.057868,25,4,1
4,3517,0.421464,3,0.757622,25,5,1


In [14]:
recs['score'] = recs['prediction']

In [15]:
recs = recs[['item', 'score', 'user','rank','RunId']]
recs.head()

,item,score,user,rank,RunId
0,53,0.633304,3,1,1
1,2839,0.556983,3,2,1
2,2503,0.543428,3,3,1
3,904,0.444571,3,4,1
4,3517,0.421464,3,5,1


Getting the predictions... (this is here for posterity, we're not actually making predictions on test set now)

In [ ]:
#preds = pd.read_parquet('my-eval/predictions.parquet')
#preds

We're going to compute per-(run,user) evaluations of the recommendations *before* combining with metadata. 

In order to evaluate the recommendation list, we need to build a combined set of truth data. Since this is a disjoint partition of users over a single data set, we can just concatenate the individual test frames:

In [16]:
truth = pd.concat((p.test for p in pairs), ignore_index = True)
truth.head()

,Unnamed: 0,user,item,rating,timestamp
0,186,3,1394,4.0,978298147
1,212,3,1196,4.0,978297539
2,229,3,2871,4.0,978297539
3,213,3,590,4.0,978297439
4,184,3,1641,2.0,978298430


In [ ]:
truth.to_csv('my-eval/truth.csv') # saving truth values to a csv for future evaluation
# truth = pd.read_csv('my-eval/truth.csv')

In [17]:
truth = truth[['user', 'item', 'rating']] # just grabbing what we need

In [18]:
truth.head() # a visual check

,user,item,rating
0,3,1394,4.0
1,3,1196,4.0
2,3,2871,4.0
3,3,590,4.0
4,3,1641,2.0


Now we can set up an analysis and compute the results.

In [19]:
rla = topn.RecListAnalysis()
rla.add_metric(topn.ndcg) # precision, recall, recip_rank, dcg, ndcg
rla.add_metric(topn.precision)
topn_compute = rla.compute(recs, truth)
topn_compute.head()

[   INFO] lenskit.topn analyzing 60400 recommendations (30200 truth rows)
[   INFO] lenskit.topn using rec key columns ['RunId', 'user']
[   INFO] lenskit.topn using truth key columns ['user']
[   INFO] lenskit.topn collecting truth data
[   INFO] lenskit.topn collecting metric results
[   INFO] lenskit.sharing.shm serialized <lenskit.topn._RLAJob object at 0x0000024EE768B400> to 1474960 pickle bytes with 12083 buffers of 2416000 bytes
[   INFO] lenskit.util.parallel setting up ProcessPoolExecutor w/ 2 workers
[   INFO] lenskit.topn measured 6040 lists in 22.52s


nrecs  ndcg  precision
RunId user                        
1     3      10.0   0.0        0.0
      4      10.0   0.0        0.0
      5      10.0   0.0        0.0
      11     10.0   0.0        0.0
      12     10.0   0.0        0.0

Next, we need to combine this with our run data, so that we know what algorithms and configurations we are evaluating:

In [20]:
topn_results = topn_compute.join(runs[['name', 'nnbrs']], on = 'RunId') # 
topn_results.head()

nrecs  ndcg  precision                name  nnbrs
RunId user                                                   
1     3      10.0   0.0        0.0  ItemKNN-CA-Average     25
      4      10.0   0.0        0.0  ItemKNN-CA-Average     25
      5      10.0   0.0        0.0  ItemKNN-CA-Average     25
      11     10.0   0.0        0.0  ItemKNN-CA-Average     25
      12     10.0   0.0        0.0  ItemKNN-CA-Average     25

We can compute the overall average performance for each algorithm configuration

In [21]:
topn_results.fillna(0).groupby(['name', 'nnbrs'])['ndcg','precision'].mean()

<ipython-input-21-a4fd14d57be5>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  topn_results.fillna(0).groupby(['name', 'nnbrs'])['ndcg','precision'].mean()


,,ndcg,precision
name,nnbrs,,
ItemKNN-CA-Average,25,0.007913,0.005563
